# spicelab — Advanced Components

This notebook demonstrates the comprehensive component library available in spicelab, including:

- **Semiconductors**: BJT, MOSFET, JFET, Zener diodes
- **Magnetic Components**: Mutual inductance, Transformers
- **Transmission Lines**: Lossless, lossy, and RC lines
- **Behavioral Sources**: Arbitrary voltage/current expressions
- **Subcircuits**: Hierarchical design with parameter passing
- **Probes**: Voltage and current measurement

Prerequisites:
- Python 3.10+
- `pip install spicelab`

In [ ]:
# Core imports
from spicelab.core.circuit import Circuit
from spicelab.core.components import (
    # Semiconductors
    JFET,
    BCurrent,
    # Behavioral Sources
    BVoltage,
    CurrentProbe,
    Inductor,
    # Magnetic
    MutualInductance,
    Resistor,
    # Subcircuits
    SubcktInstance,
    # Transmission Lines
    TLine,
    TLineLossy,
    TLineRC,
    Transformer,
    Vdc,
    # Probes
    VoltageProbe,
    ZenerDiode,
)
from spicelab.core.net import GND, Net
from spicelab.library.transistors import Bjt, Mosfet

## 1. Semiconductors

### BJT Transistors

The `Bjt` class from `spicelab.library.transistors` provides NPN and PNP bipolar junction transistors.

In [ ]:
# Create a simple common-emitter amplifier
ce_amp = Circuit("common_emitter")

# Components
vcc = Vdc("VCC", "12")
vin = Vdc("VIN", "0.7")  # DC bias
rc = Resistor("RC", "2.2k")
rb = Resistor("RB", "100k")
q1 = Bjt("Q1", "2N2222")  # NPN transistor

# Add to circuit
for comp in [vcc, vin, rc, rb, q1]:
    ce_amp.add(comp)

# Connect: VCC -> RC -> collector
ce_amp.connect(vcc.ports[0], rc.ports[0])
ce_amp.connect(rc.ports[1], q1.c)  # collector

# VIN -> RB -> base
ce_amp.connect(vin.ports[0], rb.ports[0])
ce_amp.connect(rb.ports[1], q1.b)  # base

# Emitter and grounds
ce_amp.connect(q1.e, GND)  # emitter to ground
ce_amp.connect(vcc.ports[1], GND)
ce_amp.connect(vin.ports[1], GND)

print("BJT ports:", [p.name for p in q1.ports])  # ['c', 'b', 'e']
print(ce_amp.build_netlist())

### MOSFET Transistors

The `Mosfet` class provides 4-terminal MOSFETs (drain, gate, source, bulk).

In [ ]:
# NMOS common-source amplifier
cs_amp = Circuit("common_source")

vdd = Vdc("VDD", "5")
vgs = Vdc("VGS", "2.5")
rd = Resistor("RD", "1k")
m1 = Mosfet("M1", "NMOS", "W=10u L=1u")  # With sizing params

for comp in [vdd, vgs, rd, m1]:
    cs_amp.add(comp)

# VDD -> RD -> drain
cs_amp.connect(vdd.ports[0], rd.ports[0])
cs_amp.connect(rd.ports[1], m1.d)  # drain

# Gate drive
cs_amp.connect(vgs.ports[0], m1.g)  # gate
cs_amp.connect(vgs.ports[1], GND)

# Source and bulk to ground
cs_amp.connect(m1.s, GND)  # source
cs_amp.connect(m1.b, GND)  # bulk
cs_amp.connect(vdd.ports[1], GND)

print("MOSFET ports:", [p.name for p in m1.ports])  # ['d', 'g', 's', 'b']
print(cs_amp.build_netlist())

### JFET Transistors

The `JFET` class provides junction field-effect transistors.

In [ ]:
# JFET source follower
jfet_circuit = Circuit("jfet_follower")

vdd = Vdc("VDD", "15")
rs = Resistor("RS", "2.2k")
j1 = JFET("J1", "2N5457")  # N-channel JFET

for comp in [vdd, rs, j1]:
    jfet_circuit.add(comp)

# VDD -> drain
jfet_circuit.connect(vdd.ports[0], j1.d)
jfet_circuit.connect(vdd.ports[1], GND)

# Source -> RS -> GND
jfet_circuit.connect(j1.s, rs.ports[0])
jfet_circuit.connect(rs.ports[1], GND)

# Gate to ground (self-biased)
jfet_circuit.connect(j1.g, GND)

print("JFET ports:", [p.name for p in j1.ports])  # ['d', 'g', 's']
print(jfet_circuit.build_netlist())

### Zener Diodes

The `ZenerDiode` class provides voltage reference diodes.

In [ ]:
# Simple voltage regulator
regulator = Circuit("zener_regulator")

vin = Vdc("VIN", "12")
rs = Resistor("RS", "470")
dz = ZenerDiode("DZ", "1N4733")  # 5.1V Zener
rl = Resistor("RL", "1k")

for comp in [vin, rs, dz, rl]:
    regulator.add(comp)

# VIN -> RS -> output node
out = Net("out")
regulator.connect(vin.ports[0], rs.ports[0])
regulator.connect(rs.ports[1], out)

# Zener cathode to output, anode to ground
regulator.connect(dz.c, out)  # cathode
regulator.connect(dz.a, GND)  # anode

# Load resistor
regulator.connect(rl.ports[0], out)
regulator.connect(rl.ports[1], GND)

regulator.connect(vin.ports[1], GND)

print("Zener ports:", [p.name for p in dz.ports])  # ['a', 'c']
print(regulator.build_netlist())

## 2. Magnetic Components

### Mutual Inductance

The `MutualInductance` class couples two inductors magnetically.

In [ ]:
# Coupled inductors
coupled = Circuit("coupled_inductors")

l1 = Inductor("L1", "10u")
l2 = Inductor("L2", "10u")
k1 = MutualInductance("K1", l1="L1", l2="L2", coupling=0.95)

coupled.add(l1, l2, k1)

# Connect L1
n1 = Net("n1")
coupled.connect(l1.ports[0], n1)
coupled.connect(l1.ports[1], GND)

# Connect L2
n2 = Net("n2")
coupled.connect(l2.ports[0], n2)
coupled.connect(l2.ports[1], GND)

print("MutualInductance generates:", k1.spice_card(lambda p: "0"))

### Transformer

The `Transformer` class provides an ideal transformer using coupled inductors.

In [ ]:
# 1:10 step-up transformer
xfmr_circuit = Circuit("transformer")

vac = Vdc("VAC", "10")  # AC source (simplified)
xfmr = Transformer("T1", turns_ratio=10, primary_l="1m")
rload = Resistor("RL", "10k")

xfmr_circuit.add(vac, xfmr, rload)

# Primary: VAC -> p1, p2 -> GND
xfmr_circuit.connect(vac.ports[0], xfmr.p1)
xfmr_circuit.connect(xfmr.p2, GND)
xfmr_circuit.connect(vac.ports[1], GND)

# Secondary: s1 -> load -> s2
xfmr_circuit.connect(xfmr.s1, rload.ports[0])
xfmr_circuit.connect(rload.ports[1], xfmr.s2)
xfmr_circuit.connect(xfmr.s2, GND)

print("Transformer ports:", [p.name for p in xfmr.ports])  # ['p1', 'p2', 's1', 's2']
print(xfmr_circuit.build_netlist())

## 3. Transmission Lines

Three types of transmission lines are available for signal integrity analysis.

In [ ]:
# Lossless transmission line
tline = TLine("T1", z0=50, td="1n")  # 50 ohm, 1ns delay
print("TLine ports:", [p.name for p in tline.ports])  # ['p1p', 'p1n', 'p2p', 'p2n']
print("TLine SPICE:", tline.spice_card(lambda p: p.name))

# Lossy transmission line
lossy = TLineLossy("O1", z0=75, td="2n", r=0.1, g=1e-6)
print("\nTLineLossy SPICE:", lossy.spice_card(lambda p: p.name))

# Uniform RC line
rcline = TLineRC("U1", l="1m", n=10, r=100, c="1p")
print("\nTLineRC SPICE:", rcline.spice_card(lambda p: p.name))

## 4. Behavioral Sources

Behavioral sources allow arbitrary mathematical expressions for voltage and current.

In [ ]:
# Voltage doubler using behavioral source
bv = BVoltage("B1", expr="V(in)*2")
print("BVoltage SPICE:", bv.spice_card(lambda p: {bv.p: "out", bv.n: "0"}[p]))

# Conditional voltage (comparator behavior)
bv_cmp = BVoltage("B2", expr="IF(V(ctrl)>2.5, 5, 0)")
print("Comparator SPICE:", bv_cmp.spice_card(lambda p: "node"))

# Current mirror with gain
bi = BCurrent("B3", expr="I(Vref)*10")
print("BCurrent SPICE:", bi.spice_card(lambda p: "node"))

## 5. Subcircuit Instances

The `SubcktInstance` class instantiates predefined subcircuits.

In [ ]:
# Instantiate an op-amp subcircuit
opamp = SubcktInstance("U1", subckt_name="LM741", port_names=["inp", "inn", "vcc", "vee", "out"])
print("SubcktInstance ports:", [p.name for p in opamp.ports])

# With parameters
res_sub = SubcktInstance(
    "X1", subckt_name="MYRES", port_names=["a", "b"], params={"R": "1k", "TC1": "0.001"}
)
print("With params:", res_sub.spice_card(lambda p: p.name))

## 6. Probe Components

Probes provide explicit measurement points in the circuit.

In [ ]:
# Voltage probe (creates a named net for easy measurement)
vprobe = VoltageProbe("VP1", "output_voltage")
print("VoltageProbe:", vprobe.spice_card(lambda p: "node"))

# Current probe (zero-volt source for current measurement)
iprobe = CurrentProbe("IP1")
print("CurrentProbe:", iprobe.spice_card(lambda p: "node"))
print("CurrentProbe ports:", [p.name for p in iprobe.ports])  # ['p', 'n']

## 7. Complete Example: CMOS Inverter

A practical example combining NMOS and PMOS transistors.

In [ ]:
# CMOS inverter
inv = Circuit("cmos_inverter")

# Power supply
vdd = Vdc("VDD", "3.3")
vin = Vdc("VIN", "0")  # Input

# Transistors
mp = Mosfet("MP", "PMOS", "W=2u L=0.18u")  # PMOS pull-up
mn = Mosfet("MN", "NMOS", "W=1u L=0.18u")  # NMOS pull-down

inv.add(vdd, vin, mp, mn)

# Nets
vdd_net = Net("vdd")
in_net = Net("in")
out_net = Net("out")

# VDD connections
inv.connect(vdd.ports[0], vdd_net)
inv.connect(vdd.ports[1], GND)

# PMOS: source to VDD, drain to output, bulk to VDD
inv.connect(mp.s, vdd_net)
inv.connect(mp.d, out_net)
inv.connect(mp.b, vdd_net)

# NMOS: source to GND, drain to output, bulk to GND
inv.connect(mn.s, GND)
inv.connect(mn.d, out_net)
inv.connect(mn.b, GND)

# Gates connected together (input)
inv.connect(mp.g, in_net)
inv.connect(mn.g, in_net)

# Input source
inv.connect(vin.ports[0], in_net)
inv.connect(vin.ports[1], GND)

print(inv.build_netlist())

## Summary

This notebook demonstrated the comprehensive component library in spicelab:

| Category | Components |
|----------|------------|
| Semiconductors | `Bjt`, `Mosfet`, `JFET`, `ZenerDiode` |
| Magnetic | `MutualInductance`, `Transformer` |
| Transmission Lines | `TLine`, `TLineLossy`, `TLineRC` |
| Behavioral | `BVoltage`, `BCurrent` |
| Subcircuits | `SubcktInstance` |
| Probes | `VoltageProbe`, `CurrentProbe` |

All components generate standard SPICE netlists compatible with NGSpice, LTspice, and other simulators.